## Lab: Build an Interactive Dashboard with Plotly Dash

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown( 
                                    id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value': 'ALL'},
                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                    ],
                                    placeholder='Select a Launch Site',
                                    searchable=True,
                                    # style={'width':'80%', 'padding':'3px', 'font-size': '20px',
                                    #         'text-align-last': 'center'}
                                ),                               
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):", style={'font-size': '30px'}),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min=0,
                                    max=10000,
                                    step=1000,
                                    marks={
                                        0: {'label': '0', 'style': {'font-size':'30px'}, 'margin-bottom':'10px'},
                                        2500: {'label': '2500', 'style': {'font-size':'30px'}, 'margin-bottom':'10px'},
                                        5000: {'label': '5000', 'style': {'font-size':'30px'}, 'margin-bottom':'10px'},
                                        7500: {'label': '7500', 'style': {'font-size':'30px'}, 'margin-bottom':'10px'},
                                        10000: {'label': '10000', 'style': {'font-size':'30px'}, 'margin-bottom':'10px'}
                                    },
                                    value=[min_payload, max_payload]
                                ),

                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')],
    )
def get_pie_chart(entered_site):
    # filtered_df = spacex_df

    if entered_site == 'ALL':
        # filtered_df = spacex_df.groupby(['Launch Site'], as_index=False)['class'].mean()
        filtered_df = spacex_df[spacex_df['class']==1].groupby(['Launch Site'], as_index=False)['class'].count()
        fig = px.pie(filtered_df, 
                     values='class', 
                     names='Launch Site', 
                     # title='Total Success Launches by Site'
                     )
        fig.update_traces(
            textfont_size=20,
            marker=dict(line=dict(color='#000000', width=2))
            )
        fig.update_layout(
            # showlegend=False,
            font={'size': 30},
            title={'text': '<b>Total Success Launches by Site</b>', 'font': {'size': 40}}
            )
        return fig
    else:
        # return the outcomes piechart for a selected site
        # filtered_df = spacex_df[spacex_df['Launch Site']==entered_site].groupby(['Launch Site', 'class'], as_index=False)['class'].count()
        # filtered_df = spacex_df[spacex_df['Launch Site']==entered_site].groupby(['Launch Site', 'class'], as_index=False)['class'].count()
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site].groupby(['Launch Site', 'class'], as_index=False).count()
        filtered_df = filtered_df[['Launch Site', 'class', 'Mission Outcome']].rename(columns={'Mission Outcome': 'count'}) 
        fig = px.pie(filtered_df, 
                     values='count', 
                     names='class',
                     color='class', 
                     # title=f"Total Success Launches for Site {entered_site}"
                     )
        fig.update_traces(
            textfont_size=20,
            marker=dict(line=dict(color='#000000', width=2))
            )
        fig.update_layout(
            # showlegend=False,
            font={'size': 30},
            title={'text': f"<b>Total Success Launches for Site {entered_site}</b>", 'font': {'size': 40}}
            )                     
        return fig        

In [6]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
     Input(component_id='payload-slider', component_property='value')],
    )
def get_scatter_chart(entered_site, minmax):

    print('site: ', entered_site)
    print('min, max: ', minmax)
    # print('max: ', max)

    mask = (spacex_df['Payload Mass (kg)'] >= minmax[0]) & (spacex_df['Payload Mass (kg)'] <= minmax[1])

    if entered_site == 'ALL':
        # render a scatter plot to display all values for variable Payload Mass (kg) and variable class.

        fig = px.scatter(spacex_df[mask],
                         x="Payload Mass (kg)",
                         y="class",
                         color="Booster Version Category",
                         size="Payload Mass (kg)",
                         hover_data=["Launch Site"]
                         )
        # fig.update_layout(title='Correlation between Payload and Success Launches for all Sites')
        fig.update_layout(
            # showlegend=False,
            font={'size': 30},
            title={'text': '<b>Correlation between Payload and Success Launches for all Sites</b>', 'font': {'size': 40}}
            )             
        return fig
    else:
        # filter the spacex_df first, and render a scatter chart to show

        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]

        fig = px.scatter(filtered_df[mask],
                         x="Payload Mass (kg)",
                         y="class",
                         color="Booster Version Category",
                         size='Payload Mass (kg)',
                         hover_data=["Launch Site"]
                         )
        # fig.update_layout(title=f"Correlation between Payload and Success Launches for Site {entered_site}")
        fig.update_layout(
            # showlegend=False,
            font={'size': 30},
            title={'text': f"<b>Correlation between Payload and Success Launches for Site {entered_site}</b>", 'font': {'size': 40}}
            )          
        return fig   

In [7]:
# # Uncomment to Run the app
# if __name__ == '__main__':
#     app.run_server()